<a href="https://colab.research.google.com/github/AeroEng16/FootballAnalysis/blob/main/SAHIForBallDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAHI

In [4]:
import cv2
import os

image_folder = 'D:/FootballAnalysis/FootballAnalysis/runs/predict/exp/visuals'
video_name = 'D:/FootballAnalysis/FootballAnalysis/annotatedTestvid.mp4'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
from sahi.predict import get_sliced_prediction
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict

# init any model


detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path="bestBallSoFar_0409.pt",
    confidence_threshold=0.01,
    device="cuda:0", # or 'cuda:0'
)
    
prediction = predict(
    model_type='yolov8',
    model_path="bestBallSoFar_0409.pt",
    model_device="cuda:0",
    model_confidence_threshold=0.01,
    source="testFrames",
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
    project = "ballPrediction",
    name="firstTesting",
    export_pickle=True
)

# get sliced prediction resul
# result = get_sliced_prediction(
#    "D:/FootballAnalysis/FootballAnalysis/testFrames/frame323.jpg",
#    detection_model,
#    slice_height = 256,
#    slice_width = 256,
#    overlap_height_ratio = 0.2,
#    overlap_width_ratio = 0.2
#)
#result.export_visuals(export_dir="demo_data/")

In [22]:
import pickle

dbfile = open(r"D:\FootballAnalysis\FootballAnalysis\ballPrediction\firstTesting4\pickles\frame3.pickle".replace("\\","/"),'rb')
db = pickle.load(dbfile)

for keys in db:
    print(keys)

ObjectPrediction<
    bbox: BoundingBox: <(1198.849105834961, 858.1821098327637, 1215.5352172851562, 877.3453941345215), w: 16.686111450195312, h: 19.163284301757812>,
    mask: None,
    score: PredictionScore: <value: 0.6864193081855774>,
    category: Category: <id: 0, name: ball>>
ObjectPrediction<
    bbox: BoundingBox: <(372.96112, 304.73773, 403.05533, 360.78998), w: 30.094207763671875, h: 56.05224609375>,
    mask: None,
    score: PredictionScore: <value: 0.37592369318008423>,
    category: Category: <id: 0, name: ball>>
ObjectPrediction<
    bbox: BoundingBox: <(1237.6412, 434.16074, 1266.781, 482.4332), w: 29.1397705078125, h: 48.2724609375>,
    mask: None,
    score: PredictionScore: <value: 0.3421994149684906>,
    category: Category: <id: 0, name: ball>>
ObjectPrediction<
    bbox: BoundingBox: <(783.570556640625, 369.6738739013672, 792.2418975830078, 383.21124267578125), w: 8.671340942382812, h: 13.537368774414062>,
    mask: None,
    score: PredictionScore: <value: 0.

In [4]:
from sahi.slicing import slice_coco

coco_dict, coco_path = slice_coco(
    output_coco_annotation_file_name="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/splitImages/test/labels.json",
    coco_annotation_file_path="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/test/labels.json",
    output_dir = "D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/splitImages/test/data",
    image_dir="D:/FootballAnalysis/FootballAnalysis/ballDetect4Coco/test/data",
    slice_height=256,
    slice_width=256,
    overlap_height_ratio=0.2,
    overlap_width_ratio=0.2,
)

ModuleNotFoundError: No module named 'sahi'

# Fifty One Dataset Visualisation

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io
import ast
from plotly.subplots import make_subplots

## Install FiftyOne


## Load a dataset

Let's get started by importing the FiftyOne library:

In [6]:
import fiftyone as fo

In [11]:
import fiftyone.zoo as foz

dataset = fo.Dataset.from_dir(
    dataset_dir = "C:/VSCodeProjects/FootballAnalysis/ballDetect4Coco/splitImages/train",
    dataset_type=fo.types.COCODetectionDataset,
    name="ballDataset2",
)
view = dataset.view()

print(view)

 100% |█████████████| 35604/35604 [13.1s elapsed, 0s remaining, 2.7K samples/s]      


08/27/2024 21:06:23 - INFO - eta.core.utils -    100% |█████████████| 35604/35604 [13.1s elapsed, 0s remaining, 2.7K samples/s]      


Dataset:     ballDataset2
Media type:  image
Num samples: 35604
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
View stages:
    ---


In [ ]:
print(dataset)

Now let's launch the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html) so we can explore the dataset visually. Right away you will see that because we are in a notebook, an embedded instance of the App with our dataset loaded has been rendered in the cell's output.

The [Session](https://voxel51.com/docs/fiftyone/api/fiftyone.core.session.html#fiftyone.core.session.Session) object created below is a bi-directional connection between your Python kernel and the FiftyOne App, as we'll see later.

In [13]:
session = fo.launch_app(dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.25.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



08/27/2024 21:07:00 - INFO - fiftyone.core.session.session -   
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.25.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|

